In [33]:
# dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import pandas as pd
import time
from scipy.stats import linregress

#import api key
from config import weather_api_key

#import citipy
from citipy import citipy

In [34]:
# Set up lists to hold response info
cities = []
lat_lngs = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [35]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Confirm sufficient city count
len(cities)

593

In [36]:
units = 'imperial'
# build query url
query_url = f"http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units={units}&q="

In [44]:
# create arrays to be appended
temp = []
hum = []
cloud = []
wind = []
lat = []
lon = []
cityID = []
country = []
count = 0

for city in cities:
    json_response = requests.get(query_url + city).json()
    try:
        temp.append(json_response['main']['temp_max'])
        hum.append(json_response['main']['humidity'])
        cloud.append(json_response['clouds']['all'])
        wind.append(json_response['wind']['speed'])
        lat.append(json_response['coord']['lat'])
        lon.append(json_response['coord']['lon'])
        count +=1
        print(f"{count}: Processing records for {city}")
        
        
    except KeyError:
        print(f"---Records not found for {city}---")

print("------------------------------")        
print("All cities have been processed")

# create data file for city weather data in csv file
output_data_file = "../Analysis/WeatherAPI/cities_data.csv"

1: Processing records for qaanaaq
2: Processing records for tuktoyaktuk
3: Processing records for taltal
4: Processing records for rikitea
---Records not found for saint combs---
5: Processing records for victoria
6: Processing records for hobart
7: Processing records for port alfred
8: Processing records for busselton
9: Processing records for hilo
10: Processing records for saldanha
11: Processing records for guerrero negro
12: Processing records for aketi
13: Processing records for butaritari
14: Processing records for saskylakh
15: Processing records for barrow
16: Processing records for mataura
17: Processing records for dalbandin
18: Processing records for arraial do cabo
19: Processing records for inuvik
20: Processing records for ushuaia
21: Processing records for rocha
22: Processing records for kapaa
23: Processing records for cherskiy
---Records not found for mau aima---
---Records not found for amderma---
24: Processing records for nikolskoye
25: Processing records for soui